In [23]:
import pandas as pd
from datetime import datetime, timedelta
import dateutil
import numpy as np
import scipy
import sklearn as sk
import statsmodels.api as sm
import patsy
import warnings
# Suppress all warnings
warnings.filterwarnings("ignore")

In [2]:
def preprocess(data):
    tmp = pd.to_datetime(data['Date'])
    data['Wk'] = tmp.dt.isocalendar().week
    data['Yr'] = tmp.dt.year
    data['Wk'] = pd.Categorical(data['Wk'], categories=[i for i in range(1, 53)])  # 52 weeks 
    return data

In [24]:
# What have we tried (III)

def svd(train_data, d = 5):
    # some random value
    final_data = []
    departments = train_data['Dept'].unique()

    for dept in departments:
        # Grabbing stores that have the current dept 
        filtered_train = train_data[train_data['Dept'] == dept]
        
        selected_columns = filtered_train[['Store', 'Date', 'Weekly_Sales']]
        train_dept_ts = selected_columns.pivot(index='Store', columns='Date', values='Weekly_Sales').reset_index()

        # Replace all missing values with zero
        X_train = train_dept_ts.iloc[:, 1:]
        X_train = X_train.to_numpy()
        X_train = np.nan_to_num(X_train)

        # Center X values
        store_mean = np.mean(X_train, axis=0)
        X_centered = X_train - store_mean

        # Implement SVD
        U, D, VT = np.linalg.svd(X_centered, full_matrices=False)
        
        # Take first d components and fill rest of the diag with 0s
        D[d:] = 0

        # Make a reduced rank (smoothed) version of the original dataset
        X_bar = (U @ np.diag(D) @ VT) + store_mean

        print(filtered_train)

        # need to add some logic to rebuild the train data with column labels and such
        # concat it into final_data
        # print(X_bar)
    
    return final_data
    

In [14]:
def process(train_file_path, test_file_path, pred_file_path):
    print(train_file_path)
    train = pd.read_csv(train_file_path)
    test = pd.read_csv(test_file_path)

    train_svd = svd(train)
    test_pred = pd.DataFrame()
    print(train_svd)
    
    
    train_pairs = train_svd[['Store', 'Dept']].drop_duplicates(ignore_index=True)
    test_pairs = test[['Store', 'Dept']].drop_duplicates(ignore_index=True)
    unique_pairs = pd.merge(train_pairs, test_pairs, how = 'inner', on =['Store', 'Dept'])
    
    train_split = unique_pairs.merge(train_svd, on=['Store', 'Dept'], how='left')
    train_split = preprocess(train_split)
    X = patsy.dmatrix('Weekly_Sales + Store + Dept + Yr  + Wk',
                      data = train_split,
                      return_type='dataframe')
    train_split = dict(tuple(X.groupby(['Store', 'Dept'])))
    

    test_split = unique_pairs.merge(test, on=['Store', 'Dept'], how='left')
    test_split = preprocess(test_split)
    X = patsy.dmatrix('Store + Dept + Yr  + Wk', 
                        data = test_split, 
                        return_type='dataframe')
    X['Date'] = test_split['Date']
    test_split = dict(tuple(X.groupby(['Store', 'Dept'])))
    
    keys = list(train_split)
    
    for key in keys:
        X_train = train_split[key]
        X_test = test_split[key]
     
        Y = X_train['Weekly_Sales']
        X_train = X_train.drop(['Weekly_Sales','Store', 'Dept'], axis=1)
        
        cols_to_drop = X_train.columns[(X_train == 0).all()]
        X_train = X_train.drop(columns=cols_to_drop)
        X_test = X_test.drop(columns=cols_to_drop)
     
        cols_to_drop = []
        for i in range(len(X_train.columns) - 1, 1, -1):  # Start from the last column and move backward
            col_name = X_train.columns[i]
            # Extract the current column and all previous columns
            tmp_Y = X_train.iloc[:, i].values
            tmp_X = X_train.iloc[:, :i].values
    
            coefficients, residuals, rank, s = np.linalg.lstsq(tmp_X, tmp_Y, rcond=None)
            if np.sum(residuals) < 1e-16:
                    cols_to_drop.append(col_name)
                
        X_train = X_train.drop(columns=cols_to_drop)
        X_test = X_test.drop(columns=cols_to_drop)
    
        model = sm.OLS(Y, X_train).fit()
        mycoef = model.params.fillna(0)
        
        tmp_pred = X_test[['Store', 'Dept', 'Date']]
        X_test = X_test.drop(['Store', 'Dept', 'Date'], axis=1)
        
        tmp_pred['Weekly_Pred'] = np.dot(X_test, mycoef)
        test_pred = pd.concat([test_pred, tmp_pred], ignore_index=True)
    
    result = test.merge(test_pred, on=["Store", "Dept", "Date"], how="left")
    result["Weekly_Pred"] = result["Weekly_Pred"].fillna(0)

    # Ensure we have all these colummns
    result = result[["Store", "Dept", "Date", "IsHoliday", "Weekly_Pred"]]

    result.to_csv(pred_file_path, index=False)


In [25]:
num_folds = 10

for i in range(num_folds):
    train_file_path = f'./Proj2_Data/fold_{i+1}/train.csv'
    test_file_path = f'./Proj2_Data/fold_{i+1}/test.csv'
    pred_file_path = f'./Proj2_Data/fold_{i+1}/mypred.csv'

    process(train_file_path, test_file_path, pred_file_path) 

./Proj2_Data/fold_1/train.csv
        Store  Dept        Date  Weekly_Sales  IsHoliday
0           1     1  2010-02-05      24924.50      False
1           1     1  2010-02-12      46039.49       True
2           1     1  2010-02-19      41595.55      False
3           1     1  2010-02-26      19403.54      False
4           1     1  2010-03-05      21827.90      False
...       ...   ...         ...           ...        ...
160413     45     1  2011-01-28      12774.72      False
160414     45     1  2011-02-04      15339.62      False
160415     45     1  2011-02-11      22469.46       True
160416     45     1  2011-02-18      35065.76      False
160417     45     1  2011-02-25      13886.57      False

[2520 rows x 5 columns]
        Store  Dept        Date  Weekly_Sales  IsHoliday
56          1     2  2010-02-05      50605.27      False
57          1     2  2010-02-12      44682.74       True
58          1     2  2010-02-19      47928.89      False
59          1     2  2010-02-26  

TypeError: list indices must be integers or slices, not list

In [6]:
def myeval():
    test_with_label = pd.read_csv('./Proj2_Data/test_with_label.csv')
    wae = []

    for i in range(num_folds):
        file_path = f'./Proj2_Data/fold_{i+1}/test.csv'
        test = pd.read_csv(file_path)
        test = test.drop(columns=['IsHoliday']).merge(test_with_label, on=['Date', 'Store', 'Dept'])

        file_path = f'./Proj2_Data/fold_{i+1}/mypred.csv'
        test_pred = pd.read_csv(file_path)
        test_pred = test_pred.drop(columns=['IsHoliday'])

        new_test = test.merge(test_pred, on=['Date', 'Store', 'Dept'], how='left')

        actuals = new_test['Weekly_Sales']
        preds = new_test['Weekly_Pred']
        weights = new_test['IsHoliday'].apply(lambda x: 5 if x else 1)
        wae.append(sum(weights * abs(actuals - preds)) / sum(weights))

    return wae

In [159]:
wae = myeval()
for value in wae:
    print(f"\t{value:.3f}")
print(f"{sum(wae) / len(wae):.3f}")

	2044.795
	1466.468
	1445.260
	1594.190
	2324.849
	1674.291
	1719.834
	1427.021
	1443.198
	1444.152
1658.406
